# <span style='background:Yellow'> Bottom Temp & Salt difference between ECCO and WOA

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.32.29.9:44641")
client

In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
import gcsfs
import dask
import dask.array as dsa
import numpy as np
import intake
%matplotlib inline
import cartopy.crs as ccrs #cartopy coord ref sys
import cartopy
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import regionmask
import geopandas
import scipy
import xesmf as xe

In [ ]:
xr.set_options(display_style='text')

In [ ]:
woa8110_pt = xr.open_dataset('/home/jovyan/WMT/WOA/pt_bottom_wg_8110.nc')
woa8110_pt_error = xr.open_dataset('/home/jovyan/WMT/WOA/pt_bottom_error.nc')
woa8110_slt = xr.open_dataset('/home/jovyan/WMT/WOA/san_bottom_wg_8110.nc')
woa8110_slt_error = xr.open_dataset('/home/jovyan/WMT/WOA/san_bottom_error.nc')

woa0517_pt = xr.open_dataset('/home/jovyan/WMT/WOA/pt_bottom_wg_0517.nc')
woa0517_pt_error = xr.open_dataset('/home/jovyan/WMT/WOA/pt_bottom_0517_error.nc')
woa0517_slt = xr.open_dataset('/home/jovyan/WMT/WOA/san_bottom_wg_0517.nc')
woa0517_slt_error = xr.open_dataset('/home/jovyan/WMT/WOA/san_bottom_wg_0517_error.nc')

ecco_temp = xr.open_dataset('/home/jovyan/WMT/ECCO-budgets/bottom_temp_wg.nc')
ecco_std_temp = xr.open_dataset('/home/jovyan/WMT/ECCO-budgets/bottom_std_temp_ecco.nc')
ecco_slt = xr.open_dataset('/home/jovyan/WMT/ECCO-budgets/bottom_slt_wg.nc')
ecco_std_slt = xr.open_dataset('/home/jovyan/WMT/ECCO-budgets/bottom_std_slt_ecco.nc')

surface_mask = xr.open_dataset('/home/jovyan/WMT/ECCO-budgets/surface_mask.nc')

# <span style='background:Pink'> <font color='Red'> Temperature </font>

In [ ]:
ecco_temp = ecco_temp.THETA.mean(dim='time')

### <span style='color:Brown'> Interpolating WOA (1981-2010) temp data onto ECCO's grid

In [ ]:
regridder = xe.Regridder(woa8110_pt, ecco_temp, method='bilinear')

woa8110_pt_new_grid = regridder(woa8110_pt)
woa8110_pt_new_grid

### <span style='color:Brown'> Interpolating WOA (1981-2010) temp error data onto ECCO's grid

In [ ]:
regridder_8110_error = xe.Regridder(woa8110_pt_error, ecco_temp, method='bilinear')

woa8110_pt_error_new_grid = regridder(woa8110_pt_error)
woa8110_pt_error_new_grid

## <span style='background:Yellow'> <font color='Red'> Bottom temperature `ECCO - WOA` (1981-2010) </font>

In [ ]:
#checking if nans
print(np.isnan(ecco_temp.sum()))

In [ ]:
ecco_woa8110_diff = ecco_temp - woa8110_pt_new_grid.isel(time=0).PT
ecco_woa8110_diff

In [ ]:
proj = ccrs.Mercator() #with Mercator you can put lat lon values.
fig, ax = plt.subplots(figsize=(15, 15), subplot_kw=dict(projection=proj))

ax.set_extent([-65, 30, -78, -62], crs=ccrs.PlateCarree())
ax.coastlines(resolution='50m')
#ax.add_feature(cfeature.LAND, facecolor='0.8', edgecolor='k')

ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='face',
                                        facecolor='0.5'))

im = ax.pcolormesh(ecco_woa8110_diff.lon, 
                   ecco_woa8110_diff.lat, 
                   ecco_woa8110_diff.where(surface_mask_sose.hFacC), 
                   transform=ccrs.PlateCarree(), cmap='RdBu_r', vmin=-2, vmax=2)
cb = fig.colorbar(im, ax=ax, shrink=0.4)
cb.set_label("Potential Temperature [˚C]", labelpad=12, fontsize=13)

plt.contourf(ecco_woa8110_diff.lon, 
             ecco_woa8110_diff.lat,  
             ecco_woa8110_diff.where(np.abs(ecco_woa8110_diff) < woa8110_pt_error_new_grid.PT_error[0]).where(surface_mask_sose.hFacC),
             transform=ccrs.PlateCarree(), colors='none', hatches=['////'])

g1 = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
g1.xlabels_top = False
g1.ylabels_right = False

ax.text(-0.07, 0.55, 'Latitude', va='bottom', ha='center',
        rotation='vertical', rotation_mode='anchor',
        transform=ax.transAxes, fontsize=12)
ax.text(0.5, -0.13, 'Longitude', va='bottom', ha='center',
        rotation='horizontal', rotation_mode='anchor',
        transform=ax.transAxes, fontsize=12)

plt.title("Difference between \n ECCO (1992-2015) and WOA (1981-2010)", fontsize=14);

### <span style='color:Brown'> Interpolating WOA (2005-2017) temp data onto ECCO's grid

In [ ]:
regridder_0517 = xe.Regridder(woa0517_pt, ecco_temp, method='bilinear')

woa0517_pt_new_grid = regridder_0517(woa0517_pt)
woa0517_pt_new_grid

### <span style='color:Brown'> Interpolating WOA (2005-2017) temp ERROR data onto ECCO's grid

In [ ]:
regridder_0517_error = xe.Regridder(woa0517_pt_error, ecco_temp, method='bilinear')

woa0517_pt_error_new_grid = regridder_0517_error(woa0517_pt_error)
woa0517_pt_error_new_grid

## <span style='background:Yellow'> <font color='Red'> Bottom temperature `ECCO - WOA` (2005-2017) </font>

In [ ]:
ecco_woa0517_diff = ecco_temp - woa0517_pt_new_grid.isel(time=0).PT
ecco_woa0517_diff

In [ ]:
proj = ccrs.Mercator() #with Mercator you can put lat lon values.
fig, ax = plt.subplots(figsize=(15, 15), subplot_kw=dict(projection=proj))

ax.set_extent([-65, 30, -78, -62], crs=ccrs.PlateCarree())
ax.coastlines(resolution='50m')
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='face',
                                        facecolor='0.5'))

im = ax.pcolormesh(ecco_woa0517_diff.lon, 
                   ecco_woa0517_diff.lat, 
                   ecco_woa0517_diff.where(surface_mask_sose.hFacC), 
                   transform=ccrs.PlateCarree(), cmap='RdBu_r', vmin=-2, vmax=2)
cb = fig.colorbar(im, ax=ax, shrink=0.4)
cb.set_label("Potential Temperature [˚C]", labelpad=12, fontsize=13)

plt.contourf(ecco_woa0517_diff.lon, 
             ecco_woa0517_diff.lat,  
             ecco_woa0517_diff.where(np.abs(ecco_woa0517_diff) < woa0517_pt_error_new_grid.PT_error[0]).where(surface_mask_sose.hFacC),
             transform=ccrs.PlateCarree(), colors='none', hatches=['////'])

g1 = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
g1.xlabels_top = False
g1.ylabels_right = False

ax.text(-0.07, 0.55, 'Latitude', va='bottom', ha='center',
        rotation='vertical', rotation_mode='anchor',
        transform=ax.transAxes, fontsize=12)
ax.text(0.5, -0.13, 'Longitude', va='bottom', ha='center',
        rotation='horizontal', rotation_mode='anchor',
        transform=ax.transAxes, fontsize=12)

plt.title("Difference between \n ECCO (1992-2015) and WOA (2005-2017)", fontsize=14);

# <span style='background:Pink'> <font color='Red'> Salinity </font>

In [ ]:
ecco_slt = ecco_slt.SALT.mean(dim='time')

### <span style='color:Brown'> Interpolating WOA (1981-2010) salt data onto ECCO's grid

In [ ]:
regridder_slt = xe.Regridder(woa8110_slt, ecco_slt, method='bilinear')

woa8110_slt_new_grid = regridder_slt(woa8110_slt)
woa8110_slt_new_grid

### <span style='color:Brown'> Interpolating WOA (1981-2010) salt ERROR data onto ECCO's grid

In [ ]:
regridder_slt_error = xe.Regridder(woa8110_slt_error, ecco_slt, method='bilinear')

woa8110_slt_error_new_grid = regridder_slt_error(woa8110_slt_error)
woa8110_slt_error_new_grid

## <span style='background:Yellow'> <font color='Red'> Bottom salinity `ECCO - WOA` (1981-2010) </font>

In [ ]:
#checking if nans
print(np.isnan(ecco_slt.sum()))

In [ ]:
ecco_woa8110_slt_diff = ecco_slt - woa8110_slt_new_grid.isel(time=0).san
ecco_woa8110_slt_diff

In [ ]:
proj = ccrs.Mercator() #with Mercator you can put lat lon values.
fig, ax = plt.subplots(figsize=(15, 15), subplot_kw=dict(projection=proj))

ax.set_extent([-65, 30, -78, -62], crs=ccrs.PlateCarree())
ax.coastlines(resolution='50m')
#ax.add_feature(cfeature.LAND, facecolor='0.8', edgecolor='k')

ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='face',
                                        facecolor='0.5'))
im = ax.pcolormesh(ecco_woa8110_slt_diff.lon, 
                   ecco_woa8110_slt_diff.lat, 
                   ecco_woa8110_slt_diff.where(surface_mask_sose.hFacC), 
                   transform=ccrs.PlateCarree(), cmap='RdBu_r', vmin=-1, vmax=0)
cb = fig.colorbar(im, ax=ax, shrink=0.4)
cb.set_label("Practical Salinity [psu]", labelpad=12, fontsize=13)

plt.contourf(ecco_woa8110_slt_diff.lon, 
             ecco_woa8110_slt_diff.lat, 
             ecco_woa8110_slt_diff.where(np.abs(ecco_woa8110_slt_diff) < woa8110_slt_error_new_grid.sse[0]).where(surface_mask_sose.hFacC),
             transform=ccrs.PlateCarree(), colors='none', hatches=['////'])

g1 = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
g1.xlabels_top = False
g1.ylabels_right = False

ax.text(-0.07, 0.55, 'Latitude', va='bottom', ha='center',
        rotation='vertical', rotation_mode='anchor',
        transform=ax.transAxes, fontsize=12)
ax.text(0.5, -0.13, 'Longitude', va='bottom', ha='center',
        rotation='horizontal', rotation_mode='anchor',
        transform=ax.transAxes, fontsize=12)

plt.title("Difference between \n ECCO (1992-2015) and WOA (1981-2010)", fontsize=14);

### <span style='color:Brown'> Interpolating WOA (2005-2017) salt data onto ECCO's grid

In [ ]:
regridder_0517_slt = xe.Regridder(woa0517_slt, ecco_slt, method='bilinear')

woa0517_slt_new_grid = regridder_0517_slt(woa0517_slt)
woa0517_slt_new_grid

### <span style='color:Brown'> Interpolating WOA (2005-2017) salt ERROR data onto ECCO's grid

In [ ]:
regridder_0517_slt_error = xe.Regridder(woa0517_slt_error, ecco_slt, method='bilinear')

woa0517_slt_error_new_grid = regridder_0517_slt_error(woa0517_slt_error)
woa0517_slt_error_new_grid

## <span style='background:Yellow'> <font color='Red'> Bottom salinity `ECCO - WOA` (2005-2017) </font>

In [ ]:
ecco_woa0517_slt_diff = ecco_slt - woa0517_slt_new_grid.isel(time=0).san
ecco_woa0517_slt_diff

In [ ]:
proj = ccrs.Mercator() #with Mercator you can put lat lon values.
fig, ax = plt.subplots(figsize=(15, 15), subplot_kw=dict(projection=proj))

ax.set_extent([-65, 30, -78, -62], crs=ccrs.PlateCarree())
ax.coastlines(resolution='50m')
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='face',
                                        facecolor='0.5'))

im = ax.pcolormesh(ecco_woa0517_slt_diff.lon, 
                   ecco_woa0517_slt_diff.lat, 
                   ecco_woa0517_slt_diff.where(surface_mask_sose.hFacC), 
                   transform=ccrs.PlateCarree(), cmap='RdBu_r', vmin=-1, vmax=0)
cb = fig.colorbar(im, ax=ax, shrink=0.4)
cb.set_label("Practical Salinity [psu]", labelpad=12, fontsize=13)

plt.contourf(ecco_woa0517_slt_diff.lon, 
             ecco_woa0517_slt_diff.lat, 
             ecco_woa0517_slt_diff.where(np.abs(ecco_woa0517_slt_diff) < woa0517_slt_error_new_grid.sse[0]).where(surface_mask_sose.hFacC),
             transform=ccrs.PlateCarree(), colors='none', hatches=['////'])

g1 = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
g1.xlabels_top = False
g1.ylabels_right = False

ax.text(-0.07, 0.55, 'Latitude', va='bottom', ha='center',
        rotation='vertical', rotation_mode='anchor',
        transform=ax.transAxes, fontsize=12)
ax.text(0.5, -0.13, 'Longitude', va='bottom', ha='center',
        rotation='horizontal', rotation_mode='anchor',
        transform=ax.transAxes, fontsize=12)

plt.title("Difference between \n ECCO (1992-2015) and WOA (2005-2017)", fontsize=14);